In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
from poisson_fem import PoissonFEM
import ROM
import GenerativeSurrogate as gs
import Data as dta
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import torch
from torch import optim

/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/c

In [2]:
# Some parameters
lin_dim_rom = 4                      # Linear number of rom elements
a = np.array([1, 1, 0])              # Boundary condition function coefficients
dim_z = 30                            # Latent space dimension
dtype = torch.float                  # Tensor data type
supervised_samples = {n for n in range(4)}
unsupervised_samples = {n for n in range(4, 144)}

In [3]:
# Define mesh and boundary conditions
mesh = PoissonFEM.RectangularMesh(np.ones(lin_dim_rom)/lin_dim_rom)
# mesh.plot()

def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps

def ess_boundary_fun(x):
    return 0.0
mesh.set_essential_boundary(origin, ess_boundary_fun)

def domain_boundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.set_natural_boundary(domain_boundary)

def flux(x):
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [4]:
# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
precond = ksp.getPC()
precond.setType('cholesky')
ksp.setFromOptions() #???

In [5]:
#Spepify right hand side and stiffness matrix
#Define boundary flux field
rhs = PoissonFEM.RightHandSide(mesh)
rhs.set_natural_rhs(mesh, flux)
funSpace = PoissonFEM.FunctionSpace(mesh)
K = PoissonFEM.StiffnessMatrix(mesh, funSpace, ksp)
rhs.set_rhs_stencil(mesh, K)

In [6]:
trainingData = dta.StokesData(supervised_samples, unsupervised_samples)


In [7]:
trainingData.read_data()

In [8]:
# trainingData.plotMicrostruct(1)
trainingData.reshape_microstructure_image()

In [9]:
# define rom
rom = ROM.ROM(mesh, K, rhs, trainingData.output_resolution**2)

In [10]:
model = gs.GenerativeSurrogate(rom, trainingData, dim_z)

In [11]:
steps = int(100)
for s in range(steps):
    print('step = ', s)
    batch_samples_pf = torch.multinomial(torch.ones(trainingData.n_unsupervised_samples), model.batch_size_N_unsupervised)
#     batch_samples_pc = torch.multinomial(torch.ones(trainingData.n_samples_out), model.batch_size_N_pc)
    batch_samples_pcf = torch.multinomial(torch.ones(trainingData.n_supervised_samples), model.batch_size_N_supervised)
    model.opt_latent_dist_step()
    model.pf_step(batch_samples_pf)
    model.pcf_step(batch_samples_pcf)
#     model.pc_step(batch_samples_pc)
    

step =  0
sample =  tensor([3])
diff =  tensor(3.5753e+09, grad_fn=<SumBackward0>)
step =  1
sample =  tensor([3])
diff =  tensor(nan, grad_fn=<SumBackward0>)
step =  2
sample =  tensor([1])
diff =  tensor(8.2083e+08, grad_fn=<SumBackward0>)
step =  3
sample =  tensor([3])
diff =  tensor(nan, grad_fn=<SumBackward0>)
step =  4
sample =  tensor([2])
diff =  tensor(nan, grad_fn=<SumBackward0>)
step =  5
sample =  tensor([0])
diff =  tensor(nan, grad_fn=<SumBackward0>)
step =  6


KeyboardInterrupt: 

In [ ]:
model.rom.solution.array

In [ ]:
model.rom.set_full_solution()

In [ ]:
model.rom.full_solution.array

In [ ]:
u = model.rom.mesh.interpolation_matrix @ torch.tensor(model.rom.full_solution.array, dtype=torch.float32)

In [ ]:
plt.imshow(torch.reshape(u, (129, 129)))

In [ ]:
plt.colorbar()

In [ ]:
plt.figure()

In [ ]:
plt.imshow(torch.reshape(trainingData.P[0], (129, 129)))

In [ ]:
plt.colorbar()

In [ ]:
plt.figure()

In [ ]:
plt.imshow(torch.reshape(trainingData.P[1], (129, 129)))

In [ ]:
plt.colorbar()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
from matplotlib import cm

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')

In [ ]:
X, Y = np.meshgrid(np.linspace(0, 1, 129), np.linspace(0, 1, 129))
Z = torch.reshape(trainingData.P[1], (129, 129))
surf = ax.plot_surface(X, Y, np.transpose(Z.detach().numpy()), cmap='inferno')

In [ ]:
Z2 = torch.reshape(u, (129, 129))
surf2 = ax.plot_surface(X, Y, Z2.detach().numpy())